In [1]:
import pandas as pd
import numpy as np
import re # используется для кодирования Балконов, там числа со строками и знаками вперемешку

In [2]:
#Reading CSV file
df = pd.read_csv('~/Desktop/ds_bootcamp/week3/project/data/_data.csv')
pd.set_option('display.max_columns', None)

In [3]:
#Deleting Unnecessary columns from data frame
df = df.drop(['Unnamed: 0', 'Тип'], axis=1)
df = df.drop(['Парковка'], axis=1) # Убераем парковку
df = df.drop(['Телефоны'], axis=1) # Убераем телефоны
df = df.drop(['Площадь комнат, м2'], axis=1) # Убераем площади комнат

In [4]:
#Renaming columns from Russian to English
df = df.rename(columns={'ID  объявления': 'post_id', 'Количество комнат': 'number_of_rooms', 
                            'Метро': 'metro', 'Адрес': 'address', 'Площадь, м2': 'area_m2', 'Дом': 'apt_short_desc'})

In [5]:
#Replacing nan values by the most frequent element 
df['number_of_rooms'] = df['number_of_rooms'].replace(to_replace=[None], value=df.loc[:, 'number_of_rooms'].mode()[0])

In [6]:
#Getting the number of items in list by region
lst_add = [address.split(', ')[0] for address in df['address']]
series_add = pd.Series(lst_add)
unique_counts = series_add.value_counts()
unique_counts

Москва                19737
Санкт-Петербург        3028
Краснодарский край      603
Name: count, dtype: int64

In [7]:
#Getting indexes of the rows which are not in Moscow
indexes_not_moscow = [i for i, address in enumerate(lst_add) if address != 'Москва']
indexes_not_moscow_list = indexes_not_moscow
indexes_not_moscow_list

[884,
 885,
 886,
 887,
 888,
 889,
 890,
 891,
 892,
 893,
 894,
 895,
 896,
 897,
 898,
 899,
 900,
 901,
 902,
 903,
 904,
 905,
 906,
 907,
 908,
 909,
 910,
 911,
 912,
 913,
 914,
 915,
 916,
 917,
 918,
 919,
 920,
 921,
 922,
 923,
 924,
 925,
 926,
 927,
 928,
 929,
 930,
 931,
 932,
 933,
 934,
 935,
 936,
 937,
 938,
 939,
 940,
 941,
 942,
 943,
 944,
 945,
 946,
 947,
 948,
 949,
 950,
 951,
 952,
 953,
 954,
 955,
 956,
 957,
 958,
 959,
 960,
 961,
 962,
 963,
 964,
 965,
 966,
 967,
 968,
 969,
 970,
 971,
 972,
 973,
 974,
 975,
 976,
 977,
 978,
 979,
 980,
 981,
 982,
 983,
 984,
 985,
 986,
 987,
 988,
 989,
 990,
 991,
 992,
 993,
 994,
 995,
 996,
 997,
 998,
 999,
 1000,
 1001,
 1002,
 1003,
 1004,
 1005,
 1006,
 1007,
 1008,
 1009,
 1010,
 1011,
 1012,
 1013,
 1014,
 1015,
 1016,
 1017,
 1018,
 1019,
 1020,
 1021,
 1022,
 1023,
 1024,
 1025,
 1026,
 1027,
 1028,
 1029,
 1030,
 1031,
 1032,
 1033,
 1034,
 1035,
 1036,
 1037,
 1038,
 1039,
 1040,
 1041,
 1042,
 10

In [8]:
#Replacing nan values by the most frequent element 
df['number_of_rooms'] = df['number_of_rooms'].replace(to_replace=[None], value=df.loc[:, 'number_of_rooms'].mode()[0])

In [9]:
#Creating 3 new features in data frame
df.insert(2, 'isolated_apt', 0)
df.insert(3, 'adjacent_apt', 0)
df.insert(4, 'iso_adj_apt', 0)

In [10]:
#Filling in 3 new columns by 1 if Apt has specific features related to columns 
splited_list_of_num_rooms = [i.split(', ') for i in df['number_of_rooms']]

for i in range(len(splited_list_of_num_rooms)):
    if splited_list_of_num_rooms[i][0].isdigit():
        df.loc[i, 'number_of_rooms'] = splited_list_of_num_rooms[i][0]
        
        if len(splited_list_of_num_rooms[i]) > 1:
            if splited_list_of_num_rooms[i][1] == 'Изолированная':
                df.loc[i, 'isolated_apt'] = 1
            
            if splited_list_of_num_rooms[i][1] == 'Смежная':
                df.loc[i, 'adjacent_apt'] = 1
            
            if splited_list_of_num_rooms[i][1] == 'Оба варианта':
                df.loc[i, 'iso_adj_apt'] = 1

In [11]:
#Replacing nan values by the most frequent element 
df['metro'] = df['metro'].replace(to_replace=[None], value=df.loc[:, 'metro'].mode()[0])

In [12]:
#Adding new columns related to Metro feature
df.insert(6, 'time_to_get_metro', 0)
df.insert(7, 'by_car', 0)
df.insert(8, 'by_walk', 0)

In [13]:
#Check uniqueness of transport type and create lists for future distribution
splited_list_of_metro = [metro.split(' (') for metro in df['metro']]
lst_type_of_transport_metro = []
lst_time_to_get_metro = []
lst_name_of_metro = []
for i in splited_list_of_metro:
    lst_type_of_transport_metro.append(i[1].split()[-1].rstrip(')'))
    lst_time_to_get_metro.append(i[1].split()[0])
    lst_name_of_metro.append(i[0])
    
unique_lst = pd.Series(lst_type_of_transport_metro).unique()
unique_lst

array(['пешком', 'машине'], dtype=object)

In [14]:
#Filling in 2 new columns by 1 if Metro has specific features related to columns
#And adding amount of min to get metro in 1 column
for i in range(len(splited_list_of_metro)):
    df.loc[i, 'metro'] = lst_name_of_metro[i]
    df.loc[i, 'time_to_get_metro'] = lst_time_to_get_metro[i]
    
    if lst_type_of_transport_metro[i] == 'пешком':
        df.loc[i, 'by_walk'] = 1
    else:
        df.loc[i, 'by_car'] = 1

/var/folders/db/69n9mbnx1hn26gcpf46v9d8h0000gn/T/ipykernel_89524/3928166692.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '9' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'time_to_get_metro'] = lst_time_to_get_metro[i]


In [15]:
#Creating lists of disctricts with related to them metro stations 
VAO = [
    'Авиамоторная',
    'Андроновка',
    'Белокаменная',
    'Бульвар Рокоссовского',
    'Измайлово',
    'Измайловская',
    'Локомотив',
    'Лухмановская',
    'Новогиреево',
    'Новокосино',
    'Партизанская',
    'Первомайская',
    'Перово',
    'Преображенская площадь',
    'Семеновская',
    'Соколиная Гора',
    'Сокольники',
    'Улица Дмитриевского',
    'Черкизовская',
    'Шоссе Энтузиастов',
    'Щелковская',
    'Электрозаводская',
    'Стромынка',
]

ZAO = [
    'Багратионовская',
    'Боровское шоссе',
    'Киевская',
    'Кропоткинская',
    'Крылатское',
    'Кунцевская',
    'Кутузовская',
    'Ломоносовский проспект',
    'Минская',
    'Мичуринский проспект',
    'Молодежная',
    'Новопеределкино',
    'Озёрная',
    'Озерная',
    'Парк Победы',
    'Пионерская',
    'Проспект Вернадского',
    'Рабочий Посёлок',
    'Рабочий поселок',
    'Рабочий посёлок',
    'Рабочий Поселок',
    'Раменки',
    'Сетунь',
    'Славянский бульвар',
    'Славянский бульвар',
    'Солнцево',
    'Студенческая',
    'Тропарёво',
    'Тропарево',
    'Филевский парк',
    'Фили',
    'Фили',
    'Юго-Западная',
    'Аминьевское шоссе',
    'Давыдково',
    'Дорогомиловская',
    'Кунцевская',
    'Проспект Вернадского',
    'Улица Новаторов',
]

NAO = [
    'Говорово',
    'Коммунарка',
    'Ольховая',
    'Остафьево',
    'Прокшино',
    'Рассказовка',
    'Румянцево',
    'Саларьево',
    'Филатов Луг',
    'Щербинка',
]

SAO = [
    'Аэропорт',
    'Балтийская',
    'Беговая',
    'Беломорская',
    'Бескудниково',
    'Верхние Лихоборы',
    'Водный стадион',
    'Войковская',
    'Гражданская',
    'Дегунино',
    'Динамо',
    'Дмитровская',
    'Зорге',
    'Коптево',
    'Красный Балтиец',
    'Лихоборы',
    'Марк',
    'Окружная',
    'Панфиловская',
    'Петровский парк',
    'Петровский Парк',
    'Петровско-Разумовская',
    'Покровское-Стрешнево',
    'Полежаевская',
    'Речной вокзал',
    'Савеловская',
    'Сокол',
    'Стрешнево',
    'Тимирязевская',
    'Ховрино',
    'Хорошёво',
    'Хорошево',
    'Хорошёвская',
    'Хорошевская',
    'ЦСКА',
    'Беломорская улица',
    'Петровский парк',
    'Петровско-Разумовская',
]

SVAO = [
    'Алексеевская',
    'Алтуфьево',
    'Бабушкинская',
    'Бибирево',
    'Ботанический сад',
    'Бутырская',
    'ВДНХ',
    'Владыкино',
    'Выставочный центр',
    'Лианозово',
    'Марьина роща',
    'Медведково',
    'Окружная',
    'Отрадное',
    'Ростокино',
    'Савеловская',
    'Свиблово',
    'Телецентр',
    'Улица Академика Королёва',
    'Улица академика королева',
    'Улица Академика королева',
    'Улица академика Королева',
    'Улица академика королёва',
    'Улица Академика королёва',
    'Улица академика Королёва',
    'Улица Милашенкова',
    'Улица Сергея Эйзенштейна',
    'Фонвизинская',
    'Бутырская',
    'Фонвизинская',
    'Шереметьевская',
]

SZAO = [
    'Волоколамская',
    'Митино',
    'Мнёвники',
    'Народное Ополчение',
    'Народное ополчение',
    'Октябрьское поле',
    'Пенягино',
    'Планерная',
    'Пятницкое шоссе',
    'Спартак',
    'Строгино',
    'Сходненская',
    'Трикотажная',
    'Тушинская',
    'Щукинская',
    'Нижние Мневники',
    'Терехово',
    'Улица Народного ополчения',
]

CAO = [
    'Александровский сад',
    'Арбатская',
    'Баррикадная',
    'Бауманская',
    'Беговая',
    'Белорусская',
    'Библиотека имени Ленина',
    'Боровицкая',
    'Воробьевы горы',
    'Выставочная',
    'Деловой центр',
    'Добрынинская',
    'Достоевская',
    'Каланчёвская',
    'Китай-город',
    'Комсомольская',
    'Краснопресненская',
    'Красносельская',
    'Красные ворота',
    'Крестьянская застава',
    'Кузнецкий мост',
    'Курская',
    'Лубянка',
    'Лужники',
    'Марксистская',
    'Маяковская',
    'Международная',
    'Менделеевская',
    'Новокузнецкая',
    'Новослободская',
    'Октябрьская',
    'Охотный ряд',
    'Павелецкая',
    'Парк Культуры',
    'Площадь Ильича',
    'Площадь Революции',
    'Полянка',
    'Пролетарская',
    'Проспект Мира',
    'Пушкинская',
    'Рижская',
    'Римская',
    'Серпуховская',
    'Смоленская',
    'Смоленская',
    'Спортивная',
    'Сретенский бульвар',
    'Сухаревская',
    'Таганская',
    'Тверская',
    'Театральная',
    'Тестовская',
    'Третьяковская',
    'Трубная',
    'Тургеневская',
    'Улицаа 1905 года',
    'Фрунзенская',
    'Цветной бульвар',
    'Чеховская',
    'Чистые пруды',
    'Чкаловская',
    'Шелепиха',
    'Электрозаводская',
    'Волхонка',
    'Плющиха',
    'Ржевская',
    'Рубцовская',
]

UVAO = [
    'Авиамоторная',
    'Братиславская',
    'Волгоградский проспект',
    'Волжская',
    'Выхино',
    'Депо',
    'Дубровка',
    'Жулебино',
    'Калитники',
    'Кожуховская',
    'Косино',
    'Котельники',
    'Кубанская',
    'Кузьминки',
    'Курьяново',
    'Лермонтовский проспект',
    'Лефортово',
    'Люблино',
    'Марьино',
    'Москва-Товарная',
    'Некрасовка',
    'Нижегородская',
    'Новохохловская',
    'Новохохловская',
    'Окская',
    'Перерва',
    'Печатники',
    'Рязанский проспект',
    'Стахановская',
    'Текстильщики',
    'Текстильщики',
    'Угрешская',
    'Юго-Восточная',
    'Печатники',
    'Текстильщики',
]

UZAO = [
    'Академическая',
    'Беляево',
    'Битцевский парк',
    'Бульвар адмирала Ушакова',
    'Бульвар Адмирала ушакова',
    'Бульвар адмирала ушакова',
    'Бульвар Дмитрия Донского',
    'Бунинская Аллея',
    'Бунинская аллея',
    'Бутово',
    'Калужская',
    'Каховская',
    'Коньково',
    'Нахимовский Проспект',
    'Нахимовский проспект',
    'Новоясеневская',
    'Новые Черёмушки',
    'Новые Черемушки',
    'Новые черёмушки',
    'Площадь Гагарина',
    'Профсоюзная',
    'Севастопольская',
    'Зюзино',
    'Теплый стан',
    'Теплый Стан',
    'Улица Горчакова',
    'Улица Скобелевская',
    'Улица Старокачаловская',
    'Университет',
    'Ясенево',
    'Воронцовская',
    'Каховская',
    'Севастопольский проспект',
]

UAO = [
    'Автозаводская',
    'Автозаводская',
    'Алма-Атинская',
    'Аннино',
    'Борисово',
    'Варшавская',
    'Верхние Котлы',
    'Домодедовская',
    'ЗИЛ',
    'Зябликово',
    'Кантемировская',
    'Каширская',
    'Коломенская',
    'Красногвардейская',
    'Красный Строитель',
    'Крымская',
    'Ленинский проспект',
    'Лесопарковая',
    'Москворечье',
    'Нагатинская',
    'Нагорная',
    'Орехово',
    'Покровское',
    'Пражская',
    'Технопарк',
    'Тульская',
    'Улица академика Янгеля',
    'Царицыно',
    'Царицыно',
    'Чертановская',
    'Шаболовская',
    'Шипиловская',
    'Южная',
    'Каширская',
    'Кленовый бульвар',
]

RAYONI = [
    'Аникеевка',
    'Баковка',
    'Битца',
    'Водники',
    'Долгопрудная',
    'Красногорская',
    'Лобня',
    'Мякинино',
    'мякинино',
    'Нахабино',
    'Немчиновка',
    'Новодачная',
    'Одинцово',
    'Опалиха',
    'Павшино',
    'Подольск',
    'Силикатная',
    'Сколково',
    'Хлебниково',
    'Шереметьевская',
]

In [16]:
#Creating dictionary to use it in function to distribute data into columns
region_map = {
    'vao': VAO,
    'zao': ZAO,
    'nao': NAO,
    'sao': SAO,
    'svao': SVAO,
    'szao': SZAO,
    'cao': CAO,
    'uvao': UVAO,
    'uzao': UZAO,
    'uao': UAO,
    'rayoni': RAYONI,
}

In [17]:
#Converting metro name in column to lowercase
df['metro'] = df['metro'].apply(lambda x: x.split()[-1].lower().strip())

In [18]:
#Converting metro names in districts to lowercase
for key in list(region_map.keys()):
    region_map[key.lower()] = [metro.lower() for metro in region_map.pop(key)]

In [19]:
#Function which distributes data to columns respectively 
for region, metro_list in region_map.items():
    df[region] = df['metro'].apply(lambda x: 1 if x in metro_list else 0)

In [20]:
df['metro']

0        смоленская
1        смоленская
2        смоленская
3        смоленская
4         арбатская
            ...    
23363      говорово
23364      солнцево
23365      солнцево
23366      солнцево
23367      говорово
Name: metro, Length: 23368, dtype: object

In [21]:
df['vao'].nunique()

2

In [22]:
#Updating dataframe and deleting metro and address columns, because now we are using districts
df = df.drop('metro', axis=1)
df = df.drop('address', axis=1)

In [23]:
#Replacing all type of areas in area_m2 column by total area
df['area_m2'] = df['area_m2'].apply(lambda x: x.split('/')[0])

In [24]:
#Creating new columns for apt type
df.insert(21, 'monolithic_apt', 0)
df.insert(22, 'monolithic_brick_apt', 0)
df.insert(23, 'panel_apt', 0)
df.insert(24, 'stalin_apt', 0)
df.insert(25, 'brick_apt', 0)
df.insert(26, 'old_stock_apt', 0)
df.insert(27, 'block_apt', 0)
df.insert(28, 'wooden_apt', 0)
df.insert(29, 'aerated_concrete_block_apt', 0)
df.insert(30, 'shield_apt', 0)

In [25]:
#Distributing data to the new columns
splited_list_of_apt_desc = [i.split(', ') for i in df['apt_short_desc']]
lst_of_type_of_apt = []
lst_of_floor_of_apt = []

for i in range(len(splited_list_of_apt_desc)):
    lst_of_floor_of_apt.append(splited_list_of_apt_desc[i][0])

    if len(splited_list_of_apt_desc[i]) > 1:
        lst_of_type_of_apt.append(splited_list_of_apt_desc[i][1])

In [26]:
#continue of previous step
for i in range(len(lst_of_type_of_apt)):
    if lst_of_type_of_apt[i] == 'Монолитный':
        df.loc[i, 'monolithic_apt'] = 1
            
    elif lst_of_type_of_apt[i] == 'Монолитно-кирпичный':
        df.loc[i, 'monolithic_brick_apt'] = 1
            
    elif lst_of_type_of_apt[i] == 'Панельный':
        df.loc[i, 'panel_apt'] = 1
            
    elif lst_of_type_of_apt[i] == 'Сталинский':
        df.loc[i, 'stalin_apt'] = 1
            
    elif lst_of_type_of_apt[i] == 'Кирпичный':
        df.loc[i, 'brick_apt'] = 1
            
    elif lst_of_type_of_apt[i] == 'старый фонд':
        df.loc[i, 'old_stock_apt'] = 1
            
    elif lst_of_type_of_apt[i] == 'Блочный':
        df.loc[i, 'block_apt'] = 1
            
    elif lst_of_type_of_apt[i] == 'Деревянный':
        df.loc[i, 'wooden_apt'] = 1
            
    elif lst_of_type_of_apt[i] == 'Газобетонный блок':
        df.loc[i, 'aerated_concrete_block_apt'] = 1
            
    elif lst_of_type_of_apt[i] == 'Щитовой':
        df.loc[i, 'shield_apt'] = 1

In [27]:
#Taking out the floor of the apt and placing into list
sorted_lst_of_floor_of_apt = [i.split('/')[0] for i in lst_of_floor_of_apt]

In [28]:
df.insert(31, 'apt_floor', 0)

In [29]:
for i in range(len(sorted_lst_of_floor_of_apt)):
    df.loc[i, 'apt_floor'] = sorted_lst_of_floor_of_apt[i]

/var/folders/db/69n9mbnx1hn26gcpf46v9d8h0000gn/T/ipykernel_89524/717303396.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'apt_floor'] = sorted_lst_of_floor_of_apt[i]


In [30]:
#Rid of apt_short_desc column
df = df.drop('apt_short_desc', axis=1)

In [31]:
#Looking for string None and replacing it by mode of the column
columns_to_process = ['number_of_rooms', 'time_to_get_metro', 'area_m2', 'apt_floor']

def replace_none_with_mode(df, columns):
    for column in columns:
        # Convert column to numeric, coerce errors to NaN
        df[column] = pd.to_numeric(df[column], errors='coerce')
        # Calculate the mode
        mode_value = df[column].mode()[0]
        # Replace 'None' with the mode
        df[column].fillna(mode_value, inplace=True)
    return df

df = replace_none_with_mode(df, columns_to_process)

/var/folders/db/69n9mbnx1hn26gcpf46v9d8h0000gn/T/ipykernel_89524/2663852186.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(mode_value, inplace=True)
/var/folders/db/69n9mbnx1hn26gcpf46v9d8h0000gn/T/ipykernel_89524/2663852186.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always 

In [32]:
#Converting columns to float
columns_to_convert = ['number_of_rooms', 'isolated_apt', 'adjacent_apt', 'iso_adj_apt',
                      'vao', 'zao', 'nao', 'sao', 'svao', 'szao', 'cao', 'uvao', 'uao',
                      'uzao', 'rayoni', 'time_to_get_metro', 'by_car', 'by_walk',
                      'area_m2', 'monolithic_apt', 'monolithic_brick_apt', 'panel_apt',
                      'stalin_apt', 'brick_apt', 'old_stock_apt', 'block_apt', 'wooden_apt',
                      'aerated_concrete_block_apt', 'shield_apt', 'apt_floor']

df[columns_to_convert] = df[columns_to_convert].astype(float)

In [33]:
df['time_to_subway'] = df.apply(
    lambda row: row['time_to_get_metro'] if row['by_car'] == 0 else row['time_to_get_metro'] * 2,
    axis=1
)

In [34]:
df.loc[:, 'post_id':'time_to_subway']

,post_id,number_of_rooms,isolated_apt,adjacent_apt,iso_adj_apt,time_to_get_metro,by_car,by_walk,area_m2,Цена,Описание,Ремонт,Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",monolithic_apt,monolithic_brick_apt,panel_apt,stalin_apt,brick_apt,old_stock_apt,block_apt,wooden_apt,aerated_concrete_block_apt,shield_apt,apt_floor,Лифт,Мусоропровод,Ссылка на объявление,vao,zao,nao,sao,svao,szao,cao,uvao,uzao,uao,rayoni,time_to_subway
0,271271157,4.0,0.0,0.0,0.0,9.0,0.0,1.0,200.0,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,9.0
1,271634126,4.0,0.0,0.0,0.0,8.0,0.0,1.0,198.0,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.50,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,8.0
2,271173086,4.0,0.0,0.0,1.0,7.0,0.0,1.0,200.0,"500000.0 руб./ За месяц, Залог - 500000 руб., ...","ID 36380: Шикарная 4-х км. квартира в ЖК ""Нов...",Евроремонт,NaN,На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.20,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,Пасс (1),NaN,https://www.cian.ru/rent/flat/271173086,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,7.0
3,272197456,4.0,0.0,0.0,1.0,3.0,0.0,1.0,170.0,"400000.0 руб./ За месяц, Залог - 400000 руб., ...",ID 31618: Эксклюзивное предложение. Современн...,Евроремонт,NaN,На улицу и двор,Совмещенный (3),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,3.20,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,Пасс (1),NaN,https://www.cian.ru/rent/flat/272197456,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0
4,273614615,2.0,0.0,0.0,0.0,7.0,0.0,1.0,58.0,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",Лот 111542. Татьяна Лучкина.\n\nБонус агенту 5...,Евроремонт,NaN,На улицу и двор,Совмещенный (2),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",The Book,NaN,3.90,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,"Пасс (1), Груз (1)",Да,https://www.cian.ru/rent/flat/273614615,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23363,215565511,1.0,0.0,0.0,0.0,8.0,0.0,1.0,35.0,"42000.0 руб./ За месяц, Залог - 42000 руб., Ко...",Сдаётся светлая теплая квартира (студия) с бал...,Евроремонт,Балкон (1),NaN,Совмещенный (1),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Сти...",Мещерский лес,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,"Пасс (1), Груз (1)",NaN,https://www.cian.ru/rent/flat/215565511,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
23364,274654844,1.0,0.0,0.0,0.0,7.0,0.0,1.0,38.7,"45000.0 руб./ За месяц, Залог - 45000 руб., Ко...",Сдам однокомнатную квартиру на длительный срок...,Евроремонт,Лоджия (1),Во двор,Совмещенный (1),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Сти...",Лучи,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,"Пасс (1), Груз (1)",NaN,https://www.cian.ru/rent/flat/274654844,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0
23365,268679909,2.0,0.0,0.0,1.0,6.0,0.0,1.0,43.1,"50000.0 руб./ За месяц, Залог - 50000 руб., Ко...",Рассматриваются квартиранты только славяне и т...,Дизайнерский,Балкон (1),На улицу и двор,Совмещенный (1),Можно с детьми,"Мебель на кухне, Душевая кабина, Стиральная ма...",NaN,NaN,NaN,0.0,0.0,

Bul's part

In [35]:
temp1=df['Цена']
rent_price_curr = temp1.apply(lambda x:x.split('/')[0].split())
rent_price = rent_price_curr.apply(lambda x:x[0])
# Цена аренды

In [36]:
currency = rent_price_curr.apply(lambda x:x[1]) # Валюта аренды

In [37]:
s3=temp1.apply(lambda x:x.split('Срок аренды - '))
stay_time=s3.apply(lambda x:x[1].split(',')[0]) # Виды сроков

In [38]:
deposite = temp1.apply(lambda x:x.split('Залог - '))
test1 = deposite.apply(lambda x:len(x))
bool_series1 = test1==2

deposite[bool_series1] = deposite[bool_series1].apply(lambda x:x[1].split(',')[0].split(' '))
deposite[~bool_series1] = '00'

deposite_value = deposite.apply(lambda x:x[0]) # Все залоги, если нету = то 0 строкой
deposite_currency = deposite.apply(lambda x:x[1]) # Все валюты, если нету = то 0 строкой

In [39]:
pre_pay = temp1.apply(lambda x:x.split('Предоплата ')[-1])

pre_pay_months = pre_pay.apply(lambda x:x.split(' ')[0]) 

pre_pay_bool = pre_pay.apply(lambda x:x.split(' ')[-1])
pre_pay_bool[(pre_pay_bool=='Длительный')|(pre_pay_bool=='месяцев')]='нету'
# Предоплата либо "мес", либо нету - заменил на "нету".

pre_pay_months = pre_pay_months.astype(float)
pre_pay_months[pre_pay_months>12]=0
# Количество месяцев

In [40]:
df = df.drop(['Цена'], axis=1) # Убераем цену

In [41]:
df.insert(12,'rent_price',rent_price)
df.insert(13,'rent_currency',currency)
df.insert(14,'stay_time',stay_time)
df.insert(15,'deposite',deposite_value)
df.insert(16,'deposite_currency',deposite_currency)
df.insert(17,'is_prepay',pre_pay_bool)
df.insert(18,'prepay_months',pre_pay_months)

In [42]:
df = df.drop(['Описание'], axis=1) # Убераем описание

In [43]:
df['rent_price']=df['rent_price'].astype(float)
df['deposite']=df['deposite'].astype(float)
# Переводит числовые во float

In [44]:
df = df.drop(['is_prepay'], axis=1) # Убераем логический признак
df['prepay_months'] = df['prepay_months'].astype(int) # Теперь если 0 - то нету Предоплаты, или количество месяцев предоплаты

In [45]:
df = df.rename(columns={'Ремонт': 'renovation'}) # Переименовываем колонку

In [46]:
df['renovation']=df['renovation'].fillna('Нету') # Пропуски = "Нету"

In [47]:
df = df.rename(columns={'Балкон': 'balcony'}) # Переименовываем колонку

In [48]:
df['balcony']=df['balcony'].fillna('Балкон (0), Лоджия (0)') # Пропуски = 'Балкон (0), Лоджия (0)'

In [49]:
# Здесь начинается вторая часть Второго релиза = конвертировать цены в Доллары, кодировать строки

In [50]:
df.loc[df['renovation'] == 'Нету', 'renovation'] = 'Без ремонта' # Повторяющиеся значения "Нету" и "Без ремонта" = теперь везде "Без ремонта"

In [51]:
renovation_mapping = {
    'Евроремонт': 3,
    'Дизайнерский': 4,
    'Косметический': 2,
    'Без ремонта':1,
}

df['renovation']=df['renovation'].map(renovation_mapping) # Сделали кодировку, где 1 = самый ужасный ремонт, 4 = самый лучший

In [52]:
convert_to_usd = {'руб.':0.011, '$':1, '€':1.07} # Словарь множителей от текущей валюты в Доллары
df['rent_price']=df.apply(lambda row: row['rent_price'] * convert_to_usd[row['rent_currency']], axis=1) # Переводим все цены в Доллары
df = df.drop(['rent_currency'], axis=1) # Убераем колонку валют

In [53]:
convert_to_usd_depo = {'руб.':0.011, '$':1, '€':1.07, '0':0.0} # Словарь уже для депозитов, там их кое-где нету валюты = 0
df['deposite'] = df.apply(lambda row: row['deposite'] * convert_to_usd_depo[row['deposite_currency']], axis=1) # Переводим все депозиты в Доллары
df = df.drop(['deposite_currency'], axis=1) # Убераем колонку валют

In [54]:
df['deposite']=df.apply(lambda row: row['deposite']/row['rent_price'], axis=1) # Переводим их в процентное соотношение от цены за аренду

In [55]:
stay_time_mapping = {
    'Длительный': 2,
    'На несколько месяцев': 1
}

df['stay_time']=df['stay_time'].map(stay_time_mapping) # Сделали кодировку, где 1 = на несколько месяцев, 2 = длительный срок

In [56]:
def extract_and_sum_numbers(s):
    numbers = re.findall(r'\d+', s)
    return sum(map(int, numbers))

df['balcony']=df['balcony'].apply(extract_and_sum_numbers)
# Используя regular expression я нахожу все числа и суммирую их
# В итоге Лоджия и Балкон по весу = у меня равны, а значения = это их кол-во

In [57]:
df = df.rename(columns={'balcony': 'balcony_count'}) 

In [58]:
df.iloc[:,12:18].dtypes # Все мои значения

stay_time                     int64
deposite                    float64
prepay_months                 int64
Окна                         object
Санузел                      object
Можно с детьми/животными     object
dtype: object

In [59]:
df.head(5)

,post_id,number_of_rooms,isolated_apt,adjacent_apt,iso_adj_apt,time_to_get_metro,by_car,by_walk,area_m2,renovation,balcony_count,rent_price,stay_time,deposite,prepay_months,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",monolithic_apt,monolithic_brick_apt,panel_apt,stalin_apt,brick_apt,old_stock_apt,block_apt,wooden_apt,aerated_concrete_block_apt,shield_apt,apt_floor,Лифт,Мусоропровод,Ссылка на объявление,vao,zao,nao,sao,svao,szao,cao,uvao,uzao,uao,rayoni,time_to_subway
0,271271157,4.0,0.0,0.0,0.0,9.0,0.0,1.0,200.0,4,0,5500.0,2,1.0,1,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,9.0
1,271634126,4.0,0.0,0.0,0.0,8.0,0.0,1.0,198.0,4,0,5500.0,2,1.0,1,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,8.0
2,271173086,4.0,0.0,0.0,1.0,7.0,0.0,1.0,200.0,3,0,5500.0,2,1.0,1,На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,Пасс (1),NaN,https://www.cian.ru/rent/flat/271173086,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,7.0
3,272197456,4.0,0.0,0.0,1.0,3.0,0.0,1.0,170.0,3,0,4400.0,2,1.0,1,На улицу и двор,Совмещенный (3),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,3.2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,Пасс (1),NaN,https://www.cian.ru/rent/flat/272197456,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0
4,273614615,2.0,0.0,0.0,0.0,7.0,0.0,1.0,58.0,3,0,2475.0,2,1.0,1,На улицу и двор,Совмещенный (2),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",The Book,NaN,3.9,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,"Пасс (1), Груз (1)",Да,https://www.cian.ru/rent/flat/273614615,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,7.0


Nastya's part

## Column "Можно с детьми/животными"

####  New column "Можно с животными"

In [60]:
df['pets_okay'] = df['Можно с детьми/животными'].apply(lambda x: 1 if isinstance(x, str) and 'Можно с животными' in x else (0 if isinstance(x, str) else np.nan))
mode_pets_okay = df['pets_okay'].mode()[0]
df['pets_okay'].fillna(mode_pets_okay, inplace=True)
print(df['pets_okay'])

0        1.0
1        0.0
2        0.0
3        1.0
4        0.0
        ... 
23363    0.0
23364    0.0
23365    0.0
23366    0.0
23367    1.0
Name: pets_okay, Length: 23368, dtype: float64


/var/folders/db/69n9mbnx1hn26gcpf46v9d8h0000gn/T/ipykernel_89524/2283144236.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['pets_okay'].fillna(mode_pets_okay, inplace=True)


In [61]:
percent_counts = df['pets_okay'].value_counts(normalize=True) * 100

# Print the percentage counts
print("Percentage of each value in 'pets_okay':")
print(percent_counts)

Percentage of each value in 'pets_okay':
pets_okay
0.0    69.453954
1.0    30.546046
Name: proportion, dtype: float64


####  New column "Можно с детьми"

In [62]:
df['kids_okay'] = df['Можно с детьми/животными'].apply(lambda x: 1 if isinstance(x, str) and 'Можно с детьми' in x else (0 if isinstance(x, str) else np.nan))
mode_kids_okay = df['kids_okay'].mode()[0] # 🚧
df['kids_okay'].fillna(mode_kids_okay, inplace=True)
print(df['kids_okay'])

0        1.0
1        1.0
2        1.0
3        0.0
4        1.0
        ... 
23363    1.0
23364    1.0
23365    1.0
23366    1.0
23367    1.0
Name: kids_okay, Length: 23368, dtype: float64


/var/folders/db/69n9mbnx1hn26gcpf46v9d8h0000gn/T/ipykernel_89524/1037702761.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['kids_okay'].fillna(mode_kids_okay, inplace=True)


In [63]:
percent_counts = df['kids_okay'].value_counts(normalize=True) * 100

# Print the percentage counts
print("Percentage of each value in 'kids_okay':")
print(percent_counts)

Percentage of each value in 'kids_okay':
kids_okay
1.0    98.977234
0.0     1.022766
Name: proportion, dtype: float64


## Column "Дополнительно"
#### Create columns for potentially impactful factors: Мебель в комнатах (and count, then fill NA if needed or drop the column), Кондиционер (likewise), Посудомоечная машина (likewise)

In [64]:
def create_column(df, new_col_name, phrase):
    df[new_col_name] = df['Дополнительно'].apply(
        lambda x: 1 if isinstance(x, str) and phrase in x else (0 if isinstance(x, str) else np.nan)
    )
    # Fill NaN values with the mode
    mode_value = df[new_col_name].mode()[0]
    df[new_col_name].fillna(mode_value, inplace=True)
    return df

# Create new columns
df = create_column(df, 'furniture_in_the_room', 'Мебель в комнатах')
df = create_column(df, 'air_conditioner', 'Кондиционер')
df = create_column(df, 'dishwashing', 'Посудомоечная машина')
df = create_column(df, 'fridge', 'Холодильник')
df = create_column(df, 'internet', 'Интернет')

/var/folders/db/69n9mbnx1hn26gcpf46v9d8h0000gn/T/ipykernel_89524/390896744.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[new_col_name].fillna(mode_value, inplace=True)
/var/folders/db/69n9mbnx1hn26gcpf46v9d8h0000gn/T/ipykernel_89524/390896744.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alway

In [65]:
# Checking descriptive stats for each main feature
def calculate_percentage(df, column_name):
    percent_counts = df[column_name].value_counts(normalize=True) * 100
    print(f"Percentage of each value in '{column_name}':")
    print(percent_counts)

calculate_percentage(df, 'furniture_in_the_room')
calculate_percentage(df, 'air_conditioner')
calculate_percentage(df, 'dishwashing')
# Doing reality check - these columns should largely have 1 
calculate_percentage(df, 'fridge')
calculate_percentage(df, 'internet')


print(df[['furniture_in_the_room', 'air_conditioner', 'dishwashing', 'fridge', 'internet']])


Percentage of each value in 'furniture_in_the_room':
furniture_in_the_room
1.0    92.643786
0.0     7.356214
Name: proportion, dtype: float64
Percentage of each value in 'air_conditioner':
air_conditioner
0.0    62.358781
1.0    37.641219
Name: proportion, dtype: float64
Percentage of each value in 'dishwashing':
dishwashing
0.0    66.48836
1.0    33.51164
Name: proportion, dtype: float64
Percentage of each value in 'fridge':
fridge
1.0    88.124786
0.0    11.875214
Name: proportion, dtype: float64
Percentage of each value in 'internet':
internet
1.0    80.498973
0.0    19.501027
Name: proportion, dtype: float64
       furniture_in_the_room  air_conditioner  dishwashing  fridge  internet
0                        1.0              1.0          1.0     1.0       1.0
1                        1.0              1.0          1.0     1.0       1.0
2                        1.0              1.0          1.0     1.0       1.0
3                        1.0              1.0          1.0     1.0      

## Column "Окна"
#### Create column only_street_view with supposedly negative impact


In [66]:
df['only_street_view'] = df['Окна'].apply(
    lambda x: 1 if isinstance(x, str) and 'На улицу' in x else (0 if isinstance(x, str) else np.nan)
)
mode_only_street_view = df['only_street_view'].mode()[0]
df['only_street_view'].fillna(mode_only_street_view, inplace=True)
percent_counts = df['only_street_view'].value_counts(normalize=True) * 100

print("Percentage of each value in 'only_street_view':")
print(percent_counts)


Percentage of each value in 'only_street_view':
only_street_view
0.0    74.815988
1.0    25.184012
Name: proportion, dtype: float64


/var/folders/db/69n9mbnx1hn26gcpf46v9d8h0000gn/T/ipykernel_89524/2472570807.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['only_street_view'].fillna(mode_only_street_view, inplace=True)


## Column "Высота потолков"
#### Values: transform into meters, ❗️ Remove outliers, fill NaN, consider creating ❗️ categorical levels (low, medium, high)


In [67]:
# Function to convert meters to centimeters
def convert_to_cm(value):
    try:
        value = float(value)
        if value < 10:
            return value * 100
        return value
    except ValueError:
        return np.nan


print(convert_to_cm(2.63))



263.0


In [68]:
# Apply the function to convert all values to centimeters
df['ceiling'] = df['Высота потолков, м'].apply(convert_to_cm)

print(min(df['ceiling']))

12.8


In [69]:
# Проверка
def show_unique_values(data, column_name):
    unique_values = data[column_name].dropna().unique()  # Remove NaN values before sorting
    sorted_values = sorted(unique_values, reverse=True)
    print(f"Unique values for '{column_name}' in descending order:", sorted_values)

show_unique_values(df, 'ceiling')

Unique values for 'ceiling' in descending order: [np.float64(900.0), np.float64(600.0), np.float64(580.0), np.float64(550.0), np.float64(509.99999999999994), np.float64(500.0), np.float64(470.0), np.float64(459.99999999999994), np.float64(450.0), np.float64(420.0), np.float64(415.00000000000006), np.float64(409.99999999999994), np.float64(405.0), np.float64(400.0), np.float64(392.0), np.float64(390.0), np.float64(380.0), np.float64(370.0), np.float64(360.0), np.float64(356.0), np.float64(355.0), np.float64(350.0), np.float64(341.0), np.float64(340.0), np.float64(335.0), np.float64(330.0), np.float64(325.0), np.float64(324.0), np.float64(321.0), np.float64(320.0), np.float64(315.0), np.float64(311.0), np.float64(310.0), np.float64(308.0), np.float64(307.0), np.float64(305.0), np.float64(302.0), np.float64(300.0), np.float64(299.0), np.float64(297.0), np.float64(295.0), np.float64(294.0), np.float64(293.0), np.float64(290.0), np.float64(288.0), np.float64(287.0), np.float64(286.0), np.fl

### Outliers

In [70]:
# Calculate IQR for "ceiling"
Q1 = df['ceiling'].quantile(0.25)
Q3 = df['ceiling'].quantile(0.95)
IQR = Q3 - Q1

# Define the lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Lower bound for outliers: {lower_bound}")
print(f"Upper bound for outliers: {upper_bound}")

Lower bound for outliers: 180.0
Upper bound for outliers: 404.0


In [71]:
# Identify outliers
outliers = (df['ceiling'] < lower_bound) | (df['ceiling'] > upper_bound)
print(f"Number of outliers: {outliers.sum()}")

# Calculate and print the median value for verification
median_height = df.loc[~outliers, 'ceiling'].median() #🚧 tilda for substraction
print(f"Calculated median height: {median_height}")

# Replace outliers and NaNs with the median value
df.loc[outliers, 'ceiling'] = median_height #🚧 
df['ceiling'].fillna(median_height, inplace=True)

Number of outliers: 60
Calculated median height: 264.0


/var/folders/db/69n9mbnx1hn26gcpf46v9d8h0000gn/T/ipykernel_89524/3211942022.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['ceiling'].fillna(median_height, inplace=True)


In [72]:

print(df['ceiling'].describe())

nan_count_after = df['ceiling'].isna().sum()
print(f"Number of NaN values in the 'ceiling' column after replacement: {nan_count_after}")

count    23368.000000
mean       269.332634
std         17.199493
min        200.000000
25%        264.000000
50%        264.000000
75%        264.000000
max        400.000000
Name: ceiling, dtype: float64
Number of NaN values in the 'ceiling' column after replacement: 0


### Transform into ordinal

In [73]:
# Define thresholds for categorizing ceiling heights (in centimeters) based on distribution and human comfort
low_ceiling_threshold = 259  # Uncomfortably low ceilings
high_ceiling_threshold = 300  # Spaciously high ceilings

# Print the percentage of values below the low threshold and above the high threshold
low_percentage = (df['ceiling'] < low_ceiling_threshold).mean() * 100
high_percentage = (df['ceiling'] > high_ceiling_threshold).mean() * 100
print(f"Percentage of uncomfortably low ceilings: {low_percentage}%")
print(f"Percentage of spaciously high ceilings: {high_percentage}%")

Percentage of uncomfortably low ceilings: 5.263608353303663%
Percentage of spaciously high ceilings: 4.732968161588498%


In [74]:
# Categorize ceiling heights
def categorize_ceiling(height):
    if height < low_ceiling_threshold:
        return 1  # Uncomfortably low
    elif height <= high_ceiling_threshold:
        return 2  # Medium
    else:
        return 3  # Spaciously high

# Apply the categorization
df['ceiling_ranking'] = df['ceiling'].apply(categorize_ceiling).astype(int)

# Display the distribution of the new categories
ceiling_category_distribution = df['ceiling_ranking'].value_counts()
print("Ceiling ranking distribution:")
print(ceiling_category_distribution)

Ceiling ranking distribution:
ceiling_ranking
2    21032
1     1230
3     1106
Name: count, dtype: int64


## Column "Санузел"
#### Values: need to make them numerical and categorical ("total number", "only совмещенный"), then fill NaN.


In [75]:
freq_table = df['Санузел'].value_counts(dropna=False).sort_values(ascending=False)

# Print the frequency table
print(freq_table)

Санузел
Совмещенный (1)                    10078
Раздельный (1)                      7158
NaN                                 2672
Совмещенный (2)                     1437
Совмещенный (1), Раздельный (1)      812
Раздельный (2)                       534
Совмещенный (3)                      241
Совмещенный (2), Раздельный (1)      188
Совмещенный (4)                       77
Раздельный (3)                        52
Совмещенный (1), Раздельный (2)       30
Совмещенный (3), Раздельный (1)       27
Совмещенный (2), Раздельный (2)       25
Раздельный (4)                        15
Совмещенный (3), Раздельный (3)        6
Совмещенный (4), Раздельный (1)        6
Совмещенный (4), Раздельный (2)        4
Совмещенный (1), Раздельный (3)        2
Совмещенный (2), Раздельный (3)        2
Совмещенный (2), Раздельный (4)        1
Совмещенный (3), Раздельный (2)        1
Name: count, dtype: int64


In [76]:
# Fill NaN values with the mode
mode_value = df['Санузел'].mode()[0]
df['Санузел'].fillna(mode_value, inplace=True)

/var/folders/db/69n9mbnx1hn26gcpf46v9d8h0000gn/T/ipykernel_89524/1485932565.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Санузел'].fillna(mode_value, inplace=True)


In [77]:
print(df['Санузел'].describe)

<bound method NDFrame.describe of 0                        Совмещенный (1)
1        Совмещенный (2), Раздельный (1)
2                        Совмещенный (3)
3                        Совмещенный (3)
4                        Совмещенный (2)
                      ...               
23363                    Совмещенный (1)
23364                    Совмещенный (1)
23365                    Совмещенный (1)
23366    Совмещенный (1), Раздельный (1)
23367                     Раздельный (1)
Name: Санузел, Length: 23368, dtype: object>


In [78]:
# Create "one_merged_bathroom_only" column
df['one_merged_bathroom_only'] = df['Санузел'].apply(lambda x: 1 if x == 'Совмещенный (1)' else 0)

relative_frequency = df['one_merged_bathroom_only'].value_counts(normalize=True) * 100
print("Relative frequency of values in 'one_merged_bathroom_only':")
print(relative_frequency)

Relative frequency of values in 'one_merged_bathroom_only':
one_merged_bathroom_only
1    54.561794
0    45.438206
Name: proportion, dtype: float64


In [79]:
# Function to calculate the total number of bathrooms
def calculate_total_bathrooms(value):
    if pd.isna(value):
        return 0
    total = 0
    parts = value.split(', ')
    for part in parts:
        number = int(part.split('(')[1].strip(')')) #🚧
        total += number
    return total

# Create "many_bathrooms" column
df['many_bathrooms'] = df['Санузел'].apply(lambda x: 1 if calculate_total_bathrooms(x) > 2 else 0)

# Print the DataFrame to verify the changes
print(df[['Санузел', 'one_merged_bathroom_only', 'many_bathrooms']])

                               Санузел  one_merged_bathroom_only  \
0                      Совмещенный (1)                         1   
1      Совмещенный (2), Раздельный (1)                         0   
2                      Совмещенный (3)                         0   
3                      Совмещенный (3)                         0   
4                      Совмещенный (2)                         0   
...                                ...                       ...   
23363                  Совмещенный (1)                         1   
23364                  Совмещенный (1)                         1   
23365                  Совмещенный (1)                         1   
23366  Совмещенный (1), Раздельный (1)                         0   
23367                   Раздельный (1)                         0   

       many_bathrooms  
0                   0  
1                   1  
2                   1  
3                   1  
4                   0  
...               ...  
23363          

## Column "Лифт"
#### Create columns: "has_elevator", "has_cargo_elevator"


In [80]:
# Step 1: Create the `no_elevators` column
df['no_elevators'] = df['Лифт'].isna().astype(int)

# Step 2: Create the `has_cargo_elevator` column
df['has_cargo_elevator'] = df['Лифт'].apply(lambda x: 1 if isinstance(x, str) and 'Груз' in x else 0)

# Print the DataFrame to verify the changes
print(df[['Лифт', 'no_elevators', 'has_cargo_elevator']])

# Print the relative frequency of values
print("Relative frequency of 'no_elevators':")
print(df['no_elevators'].value_counts(normalize=True) * 100)

print("Relative frequency of 'has_cargo_elevator':")
print(df['has_cargo_elevator'].value_counts(normalize=True) * 100)

                     Лифт  no_elevators  has_cargo_elevator
0      Пасс (4), Груз (1)             0                   1
1      Пасс (1), Груз (1)             0                   1
2                Пасс (1)             0                   0
3                Пасс (1)             0                   0
4      Пасс (1), Груз (1)             0                   1
...                   ...           ...                 ...
23363  Пасс (1), Груз (1)             0                   1
23364  Пасс (1), Груз (1)             0                   1
23365                 NaN             1                   0
23366            Пасс (3)             0                   0
23367            Пасс (3)             0                   0

[23368 rows x 3 columns]
Relative frequency of 'no_elevators':
no_elevators
0    76.46354
1    23.53646
Name: proportion, dtype: float64
Relative frequency of 'has_cargo_elevator':
has_cargo_elevator
0    71.597912
1    28.402088
Name: proportion, dtype: float64


## Column "Мусоропровод"
##### fill NaN with moda



In [81]:
def show_unique_values(data, column_name):
    unique_values = data[column_name].dropna().unique()  # Remove NaN values before sorting
    sorted_values = sorted(unique_values, reverse=True)
    print(f"Unique values for '{column_name}' in descending order:", sorted_values)

# for 'Мусоропровод'
show_unique_values(df, 'Мусоропровод')

Unique values for 'Мусоропровод' in descending order: ['Нет', 'Да']


In [82]:
# 🚧 does not work for some reason, timebox for investigation is over
# #fillNa
# mode_garbage = df['Мусоропровод'].mode()
# print(mode_garbage)
# df['Мусоропровод'].fillna(mode_garbage, inplace=True)

# # df['garbage'] = df['Мусоропровод'].map({'Да': 1, 'Нет': 0})



# # # Step 4: Ensure the column type is integer
# # df['garbage'] = df['garbage'].astype(int)

# # # # Step 5: Tests
# # # relative_frequency = df['garbage'].value_counts(normalize=True) * 100
# # # print("Relative frequency of values in 'garbage':")
# # # print(relative_frequency)
# print(df['Мусоропровод'].describe())


In [83]:
print(df.dtypes)


post_id                       int64
number_of_rooms             float64
isolated_apt                float64
adjacent_apt                float64
iso_adj_apt                 float64
                             ...   
ceiling_ranking               int64
one_merged_bathroom_only      int64
many_bathrooms                int64
no_elevators                  int64
has_cargo_elevator            int64
Length: 62, dtype: object


In [84]:
#Getting rid of columns 
df = df.drop(['Окна'], axis=1)
df = df.drop(['Можно с детьми/животными'], axis=1)
df = df.drop(['Дополнительно'], axis=1)
df = df.drop(['Название ЖК'], axis=1)
df = df.drop(['Серия дома'], axis=1)
df = df.drop(['Ссылка на объявление'], axis=1)
df = df.drop(['ceiling'], axis=1)
df = df.drop(['internet'], axis=1)
df = df.drop(['fridge'], axis=1)
df = df.drop(['Санузел'], axis=1)
df = df.drop(['Высота потолков, м'], axis=1)
df = df.drop(['Лифт'], axis=1)
df = df.drop(['Мусоропровод'], axis=1)
df = df.drop(['time_to_get_metro'], axis=1)
df = df.drop(['by_car'], axis=1)
df = df.drop(['by_walk'], axis=1)

In [85]:
df.head(5)

,post_id,number_of_rooms,isolated_apt,adjacent_apt,iso_adj_apt,area_m2,renovation,balcony_count,rent_price,stay_time,deposite,prepay_months,monolithic_apt,monolithic_brick_apt,panel_apt,stalin_apt,brick_apt,old_stock_apt,block_apt,wooden_apt,aerated_concrete_block_apt,shield_apt,apt_floor,vao,zao,nao,sao,svao,szao,cao,uvao,uzao,uao,rayoni,time_to_subway,pets_okay,kids_okay,furniture_in_the_room,air_conditioner,dishwashing,only_street_view,ceiling_ranking,one_merged_bathroom_only,many_bathrooms,no_elevators,has_cargo_elevator
0,271271157,4.0,0.0,0.0,0.0,200.0,4,0,5500.0,2,1.0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,9.0,1.0,1.0,1.0,1.0,1.0,0.0,2,1,0,0,1
1,271634126,4.0,0.0,0.0,0.0,198.0,4,0,5500.0,2,1.0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,8.0,0.0,1.0,1.0,1.0,1.0,1.0,3,0,1,0,1
2,271173086,4.0,0.0,0.0,1.0,200.0,3,0,5500.0,2,1.0,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,7.0,0.0,1.0,1.0,1.0,1.0,1.0,3,0,1,0,0
3,272197456,4.0,0.0,0.0,1.0,170.0,3,0,4400.0,2,1.0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,1.0,1.0,1.0,1.0,3,0,1,0,0
4,273614615,2.0,0.0,0.0,0.0,58.0,3,0,2475.0,2,1.0,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,7.0,0.0,1.0,1.0,0.0,1.0,1.0,3,0,0,0,1


In [86]:
print(df.describe(include='all'))

            post_id  number_of_rooms  isolated_apt  adjacent_apt  \
count  2.336800e+04     23368.000000  23368.000000  23368.000000   
mean   2.676265e+08         1.968718      0.278201      0.031924   
std    1.876862e+07         1.000153      0.448123      0.175802   
min    1.072986e+08         1.000000      0.000000      0.000000   
25%    2.712663e+08         1.000000      0.000000      0.000000   
50%    2.739030e+08         2.000000      0.000000      0.000000   
75%    2.746872e+08         3.000000      1.000000      0.000000   
max    2.750064e+08         6.000000      1.000000      1.000000   

        iso_adj_apt       area_m2    renovation  balcony_count    rent_price  \
count  23368.000000  23368.000000  23368.000000   23368.000000  23368.000000   
mean       0.079553     62.819425      2.534620       0.753423    924.088734   
std        0.270606     46.699425      0.892473       0.636248   1410.673243   
min        0.000000      7.000000      1.000000       0.000000    1

In [87]:
#Deleting rows which are not in Moscow
df = df.drop(index=indexes_not_moscow_list)
df = df.reset_index(drop=True)

In [88]:
#Saving dataframe to CSV file
df.to_csv('../data/data.csv')